# Probabilistic Record Linkage in Python #

## Overview:

While working with data, we often encounter the need to merge or link records across different datasets that don't have a consistent, unique key. This challenge can be addressed using techniques commonly known as probabilistic matching or record linkage.

The essence of record linkage is to identify pairs of records that refer to the same entities (like individuals, companies, locations, etc.) across different data sources. It's particularly useful when the datasets lack unique keys or identifiers, or when some identifiers are missing or inconsistent across the sets.

## Real-World Applications:

(1) **Data de-duplication**: Identifying and removing duplicates within a dataset, such as a customer who creates multiple accounts with inconsistent entries on an e-commerce platform; (2) **Merging datasets**: combining datasets without a common key or identifier, for example, integrating patient records from multiple hospitals that don't use a common patient ID or integrating birth certificates from notaries with birth records from hospitals; (3) **Data cleaning**: Addressing issues of data inconsistency, like varied spellings of names or addresses.

## This Tutorial:

In this section, we'll be working with two county-by-year datasets. The datasets have different estimates of the same variables as well as varying county and state name spellings (including typos). We'll illustrate how to link them effectively.


Both datasets include the name of the county, state name, estimated unemployment rate, estimated number of unemployment insurance filings per capita, and the log of the estimated total population. The challenge lies on the different estimates for each variable and inconsistencies in naming conventions. Since this data was derived from real data, we possess the true key (FIPS code) to validate our linkage outcomes.

We will look at some applications of machine learning techniques to record linkage. In essence, when we have unique identifiers for some observations but not for others, we can leverage the precisely linked ones to train a machine learning model. Then, this trained model can be deployed to predict and identify potential matches for data points lacking definitive keys.

Our first step will be importing Libraries and loading the data. Be sure to install all the libraries beforehand.

In [296]:
from recordlinkage.base import BaseCompareFeature
from recordlinkage.preprocessing import clean
from recordlinkage.index import Block
import recordlinkage as rl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import numpy as np

## Loading the data: 

Let's proceed to load our datasets. Given that I intentionally introduced incorrect characters to names, there's a possibility that some may observations present encoding errors. To circumvent this, we'll opt to ignore these problematic characters and let our linkage process handle the issue. For the sake of this demonstration and to reduce the computational burden, we will limit our data to the years 2011 and 2012. This ensures our examples run quickly. To clearly distinguish the source of each variable, we'll modify the column names accordingly.

In [297]:
with open('C:/GitHub/record_linkage_counties_1.csv', 'r', encoding='utf-8', errors='ignore') as file:
    df1 = pd.read_csv(file)
with open('C:/GitHub/record_linkage_counties_2.csv', 'r', encoding='utf-8', errors='ignore') as file:
    df2 = pd.read_csv(file)
df1 = df1[(df1['year'] == 2012) | (df1['year'] == 2011)]
df2 = df2[(df2['year'] == 2012) | (df2['year'] == 2011)]

In [298]:
df1.rename(columns={'county_name':'county_name_1','state_name':'state_name_1','unemp_rate':'unemp_rate_1','ui_pc':'ui_pc_1','lpop':'lpop_1'}, inplace=True)
df2.rename(columns={'county_name':'county_name_2','state_name':'state_name_2','unemp_rate':'unemp_rate_2','ui_pc':'ui_pc_2','lpop':'lpop_2'}, inplace=True)

## Data Inspection:

Let's take a moment to review a few rows from each dataset. It's essential to note that the rows are not aligned across the two datasets; in other words, the i-th row in the first dataset does not necessarily correspond to the i-th row in the second dataset. Additionally, observe the typos present in the county and state names. We also have the true FIPS code available, which, in conjunction with the year, will be utilized later to evaluate the accuracy of our linkage efforts.

In [299]:
df1

,year,State_CountyFIPS,ui_pc_1,unemp_rate_1,state_name_1,county_name_1,lpop_1
4,2011,51650,0.254480,7.984761,Virginir,"aampton city, VA",11.885170
11,2011,55131,0.421976,7.950506,cisconsin,"aashington County, Wisconsin",11.814306
12,2011,49011,0.110855,7.195126,Utas,aavis,12.673280
36,2012,8001,0.116782,10.373718,Colohado,"Adams Copnty, CO",13.029529
47,2011,8001,0.147219,10.544022,Coloiado,Adams Countyt CO,13.033819
...,...,...,...,...,...,...,...
11277,2011,4027,0.301561,25.002157,rrizona,Yuma,12.290334
11304,2011,13021,0.031851,10.022853,GA,zibb,12.005750
11310,2012,17201,0.506335,9.452187,Ixlinois,"zinnebago County, IL",12.649313
11318,2012,12999,0.074315,9.652677,Fyorida,"znidentified Counties, FL",25.030134


In [300]:
df2

,year,State_CountyFIPS,ui_pc_2,unemp_rate_2,state_name_2,county_name_2,lpop_2
0,2011,1003,0.091076,8.569021,xlabama,Baldwln,12.164355
1,2012,1003,0.089274,9.072143,Alybama,Bsldwin,12.151860
18,2012,1015,0.050049,10.310761,Alabxma,Calnoun,11.756068
34,2011,1015,0.077642,11.201981,Alalama,Calhoub,11.743222
38,2012,1055,0.071865,10.013217,Acabama,"Etowxh County, AL",11.555716
...,...,...,...,...,...,...,...
11289,2012,55139,0.405994,6.094930,Wisconsvn,Winnebago Countyf WI,12.118357
11300,2011,55999,0.424530,7.017834,Wiscynsin,"Unidentifies Counties, WI",25.483953
11305,2012,55999,0.276715,5.611607,Winconsin,"Unidenoified Counties, WI",25.437571
11316,2012,56999,0.020655,6.066435,WY,"Unidehtified Counties, WY",24.045418


## Creating and Evaluating Candidate Links:

We start by merging our datasets using the true FIPS code. This gives us a set of known matches. Then, we split the data into a training set and a validation set. This is similar to using observations with precise keys to train a model, and then applying this model to other data. The validation set will serve as our ground truth to assess model accuracy later on.

Next, we create candidate links with Record Linkage Toolkit. With our datasets prepared, we generate all possible pairs of records across them. This step identifies potential links or matches between records in the two datasets. Once we have our candidate pairs, we need to measure how similar they are. That is, for each candidate pair, we will calculate a set of similarity indices. 

Similarity calculations are done using different indices based on the type of each variable. For numerical data, we might apply a function to the difference between values so that numbers within a certain threshold are treated as equivalent. For string data, we apply methods like the Levenshtein distance. At their core, these metrics count the number of edits needed to transform one string into another (e.g., character additions, deletions, or substitutions). We'll utilize three distinct similarity measures for state and county names, capturing the all the nuances of potential differences.

Important to note, the total number of potential pairs for many observations can be too large, rendering the process computationally challenging or even infeasible. To manage this, we utilize a technique commonly referred to in the record linkage literature as 'blocking'. In essence, blocking narrows down the pairs we consider by requiring an exact match for certain "blocked" variables. In our case, we block by 'Year', meaning we only consider pairs that share the exact same value for the 'Year' variable. A common practice in record linkage is to block based on geographic regions, such as only pairing records within the same state. While this method may introduce some inaccuracies, it's often a necessary compromise to ensure computational efficiency.

With our similarity indices in hand, we're ready to introduce machine learning. We use support vector machines (SVMs) and naive bayes (NB) for this task. The SVM aims to find the optimal boundary (or hyperplane) that separates the candidate pairs into true matches and mismatches, while the NB method evaluates the conditional probability of a match being genuine based on particular sets of similarity indices. That is, Support Vector Machines (SVMs) work by assigning specific parameters to the similarity indices, with the aim of constructing a hyperplane that best separates the candidate pairs into genuine matches and mismatches. The model fine-tunes these parameters or weights through iterative processes, adjusting based on penalties assigned for narrow or wrong classifications. On the other hand, Naive Bayes (NB) estimates the likelihood of a pair being a true match based on specific observed similarity scores. It evaluates the conditional probability of a match being genuine, given its specific similarity profile. If such a profile is found elsewhere, we can recover the calculated conditional probability.

After training our models on the dataset with known matches (i.e., those with FIPS codes), we can assess their performance on the validation set. This set comprises pairs whose true match status is known but hasn't been exposed to the model during training. In essence, our trained model predicts the likelihood of a candidate pair being a true match, even when FIPS codes are absent. By averaging the predictions from three distinct models, we enhance the robustness of our match decisions. At its core, machine learning in this context refines how we weigh each similarity index based on previously linked data. It offers guidance on the importance of each index in identifying a genuine match. 

Finally, we use our validation set to assess the effectiveness of our machine learning model. It allows us to gauge how well our trained model generalizes to new, unseen data.

Therefore, our process involves (1) identifying all possible record pairs, (2) calculating how similar they are, using several similarity measures, and (3) leveraging machine learning to translate similarity measures into true matches. In essence, machine learning leverages previously linked data to determine the importance of each similarity index. By analyzing the linked data, it finds weights corresponding to the relevance of every similarity score in obtaining a true match. In the absence of machine learning or linked data, we'd resort to setting arbitrary thresholds for similarity scores, which would then determine if a pair qualifies as a genuine link or match.


In [301]:
df = df1.merge(df2, on=['State_CountyFIPS', 'year'], suffixes=('_1', '_2'))
df_train, df_val = train_test_split(df, test_size=0.5, random_state=45)

In [302]:
def generate_candidate_links(df1,df2):
    indexer = rl.Index()
    indexer.add(Block('year','year'))
    candidate_links = indexer.index(df1,df2)
    comparer = rl.Compare()
    comparer.string('county_name_1','county_name_2',method='damerau_levenshtein',threshold=0.80,label='county_name_dl')
    comparer.string('county_name_1','county_name_2',method='jarowinkler',label='county_name_j')
    comparer.string('county_name_1','county_name_2', method='levenshtein',label='county_name_l')
    comparer.string('state_name_1','state_name_2',method='damerau_levenshtein',threshold=0.80,label='state_name_dl')
    comparer.string('state_name_1','state_name_2',method='jarowinkler',label='state_name_j')
    comparer.string('state_name_1','state_name_2', method='levenshtein',label='state_name_l')
    comparer.numeric('unemp_rate_1', 'unemp_rate_2',label='unemp')
    comparer.numeric('ui_pc_1', 'ui_pc_2',label='ui')
    comparer.numeric('lpop_1', 'lpop_2',label='lpop')
    comparer.exact('State_CountyFIPS','State_CountyFIPS',label='fips')
    feature_vectors = comparer.compute(candidate_links, df1, df2)
    return feature_vectors

In [303]:
df_train1 = df_train[['county_name_1','state_name_1','unemp_rate_1','ui_pc_1','lpop_1','State_CountyFIPS','year']]
df_train2 = df_train[['county_name_2','state_name_2','unemp_rate_2','ui_pc_2','lpop_2','State_CountyFIPS','year']]
train_feature_vectors = generate_candidate_links(df_train1,df_train2)

In [304]:
df_val1 = df_val[['county_name_1','state_name_1','unemp_rate_1','ui_pc_1','lpop_1','State_CountyFIPS','year']]
df_val2 = df_val[['county_name_2','state_name_2','unemp_rate_2','ui_pc_2','lpop_2','State_CountyFIPS','year']]
validation_feature_vectors = generate_candidate_links(df_val1,df_val2)

In [305]:
def classify_links(Xt,yt,xp):
    clf_perceptron = Perceptron()
    clf_perceptron.fit(Xt,yt)
    y_pred_perceptron = clf_perceptron.predict(xp)
    clf_gnb = GaussianNB()
    clf_gnb.fit(Xt, yt)
    y_pred_gnb = clf_gnb.predict(xp)
    clf_svc = SVC()
    clf_svc.fit(Xt, yt)
    y_pred_svc = clf_svc.predict(xp)    
    mean_pred = (y_pred_perceptron + y_pred_svc + y_pred_gnb)/3
    output = np.where(mean_pred >= 0.5, 1, 0)
    return output

In [306]:
Xt = train_feature_vectors.drop(columns=['fips'])
yt = train_feature_vectors['fips']
xp = validation_feature_vectors.drop(columns=['fips'])

In [307]:
y_pred = classify_links(Xt,yt,xp)
validation_feature_vectors['fips_pred'] = y_pred
accuracy = accuracy_score(validation_feature_vectors['fips'], validation_feature_vectors['fips_pred'])
print("Accuracy:", accuracy)

Accuracy: 0.9989602943477361


## Applying Trained Models

With our model trained, we find an impressive accuracy of approximately 0.999. However, it's vital to interpret this correctly, since it is misleading. In the context of record linkage, this accuracy encompasses all possible pairs, meaning many obvious non-matches that are rightly discarded. But when we focus at the proportion of true pairs accurately identified by the model, the calculated accuracy tends to diminish.

To demonstrate, we'll apply our trained model to the entirety of our two datasets, pretending for a moment that the FIPS codes don't exist. Using our identified pairs, we can generate a linking ID, acting as a replacement for the FIPS code, to merge the datasets. Once merged and sorted, we can visualize the results. At first glance, we spot numerous accurate matches, but wrong ones are evident too. For example, unidentified counties in Florida are inaccurately paired with those in Arkansas. Similarly, we observe Merrimack, New Hampshire, incorrectly matched with Strafford, New Hampshire. Overall, our trained model accurately links about 76.39% of all possible correct matches that would be achieved with a unique FIPS code. It's worth noting that this accuracy might drop further when applied to entirely different datasets since part of our existing data informed the model's training. Nevertheless, in scenarios where this is the best available approach, there's room to enhance accuracy by leveraging more advanced machine learning techniques and similarity metrics.

In [312]:
candidate_links = generate_candidate_links(df1,df2)
candidate_links['paired'] = classify_links(Xt,yt,candidate_links.drop(columns=['fips']))

In [313]:
pairs = candidate_links[candidate_links['paired'] == 1]
linked_df1 = df1.reindex(paired_df.index.get_level_values(0))
linked_df1['link_id'] = range(1, len(linked_df1) + 1)
linked_df2 = df2.reindex(paired_df.index.get_level_values(1))
linked_df2['link_id'] = range(1, len(linked_df2) + 1)

In [314]:
linked_df1

,year,State_CountyFIPS,ui_pc_1,unemp_rate_1,state_name_1,county_name_1,lpop_1,link_id
4,2011,51650,0.254480,7.984761,Virginir,"aampton city, VA",11.885170,1
12,2011,49011,0.110855,7.195126,Utas,aavis,12.673280,2
47,2011,8001,0.147219,10.544022,Coloiado,Adams Countyt CO,13.033819,3
68,2011,42001,0.492924,7.551140,Pennsylvania,Ahams,11.581802,4
77,2011,45003,0.120590,9.405848,South Carulina,"Aiken County,qSouth Carolina",12.026858,5
...,...,...,...,...,...,...,...,...
11310,2012,17201,0.506335,9.452187,Ixlinois,"zinnebago County, IL",12.649313,1175
11318,2012,12999,0.074315,9.652677,Fyorida,"znidentified Counties, FL",25.030134,1176
11318,2012,12999,0.074315,9.652677,Fyorida,"znidentified Counties, FL",25.030134,1177
11327,2012,33017,0.085006,5.278882,New Hagpshire,"ztrafford County, NH",11.802237,1178


In [267]:
linked_df2

,year,State_CountyFIPS,ui_pc_2,unemp_rate_2,state_name_2,county_name_2,lpop_2,link_id
10618,2011,51650,0.235620,9.220591,Virginis,"Hamqton city, Virginia",11.845647,1
10301,2011,49011,0.113033,7.198372,Uteh,Davir,12.708097,2
1251,2011,8001,0.148836,10.562110,Crlorado,Adamn,13.087878,3
8331,2011,42001,0.487089,7.536445,Pennsdlvania,"Adams County, Pennsylvania",11.563190,4
8988,2011,45003,0.116630,9.425247,South rarolina,tiken,11.989864,5
...,...,...,...,...,...,...,...,...
3255,2012,17201,0.443382,8.462302,Illinous,"Winnebago Codnty, IL",12.609329,1175
580,2012,5999,0.159035,8.785788,arkansas,"Unidentified Counties, AR",25.131620,1176
2283,2012,12999,0.097206,10.472486,Florina,"Unidkntified Counties, FL",25.092495,1177
5895,2012,33013,0.122031,5.054955,NewkHampshire,"Merrimack County, NH",11.947586,1178


In [323]:
linked_df = linked_df1.merge(linked_df2, on=['link_id'], suffixes=('_1', '_2'))
correct_links = linked_df[linked_df['State_CountyFIPS_1'] == linked_df['State_CountyFIPS_2']].shape[0]
total_links = df1.shape[0]

In [324]:
print("True accuracy:"+str(correct_links/total_links))

True accuracy:0.7639109697933227
